# Spacy text to sql

In [ ]:
import spacy
from openai import OpenAI



# Load spaCy model
nlp = spacy.load("en_core_web_md")  # 'md' model provides vectors for similarity

# Define schema terms
schema = {
    "customers": ["customer_id", "name", "email", "location"],
    "orders": ["order_id", "customer_id", "product_id", "quantity", "order_date"],
    "products": ["product_id", "name", "price"]
}

# Flatten the schema for matching
schema_terms = []
for table, columns in schema.items():
    schema_terms.append(table)
    schema_terms.extend([f"{table}.{col}" for col in columns])


In [12]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

# Access the OpenAI key
openai_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()


In [13]:
def get_relevant_items_spacy(query, top_k=3):
    # Parse query with spaCy
    query_doc = nlp(query)

    # Calculate similarity between query and schema terms
    relevance_scores = []
    for term in schema_terms:
        schema_doc = nlp(term)
        score = query_doc.similarity(schema_doc)
        relevance_scores.append((term, score))
    
    # Sort terms by relevance score
    relevance_scores = sorted(relevance_scores, key=lambda x: x[1], reverse=True)
    
    # Get top-k relevant terms
    relevant_items = relevance_scores[:top_k]
    return relevant_items

In [16]:
def generate_sql_query_spacy(query):
    # Get top relevant schema items
    relevant_items = get_relevant_items_spacy(query)
    
    # Construct prompt with relevant schema items
    relevant_schema_text = "\n".join([f"- {item[0]}" for item in relevant_items])
    prompt = (
        f"Given the following SQL database schema and the query request, generate an SQL query.\n\n"
        f"Schema:\n{relevant_schema_text}\n\n"
        f"Query request: {query}\n\n"
        "SQL Query:"
    )

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt},
            {
                "role": "user",
                "content": "Write sql for developer"
            }
        ]
    )

    # Extract and return SQL query from response
    sql_query = completion.choices[0].message
    return sql_query


In [17]:
# Example query
query = "Show me all orders placed by customers in New York"
sql_query = generate_sql_query_spacy(query)
print("Generated SQL Query:", sql_query)

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_5520/139854315.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  score = query_doc.similarity(schema_doc)


Generated SQL Query: ChatCompletionMessage(content="Here’s an SQL query to show all orders placed by customers in New York:\n\n```sql\nSELECT\n    o.* \nFROM\n    orders o\nJOIN\n    customers c ON o.customer_id = c.customer_id\nWHERE\n    c.city = 'New York';\n```\n\nIn this query:\n- We're selecting all columns from the `orders` table (`o.*`).\n- We're joining the `orders` table with the `customers` table on the `customer_id` field.\n- We're filtering the results to only include customers whose city is 'New York'.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
